
________________________________________________________________________________________________________________________________

________________________________________________________________________________________________________________________________


# Modèle d'analyse de sentiments - Sentiment analysis model


________________________________________________________________________________________________________________________________

________________________________________________________________________________________________________________________________

## Sujet - Subject

Le but de ce notebook est de créer un modèle qui prédit le sentiment (positif/négatif) d'un texte donné. Pour cela, nous nous appuyons sur une base de données contenant environ 5700 commentaires de livres et leur sentiment.
> 
*
The purpose of this notebook is to create a model which can predict the feeling (positive/negative) of a text.
For that, we will use a database with more than 5700 comments of books and their feeling.
*

L'algorithme utilisé dans notre cas est la régression logistique. Cependant, vous verrez que le nombre de variables, correspondant aux mots utilisés, est très important. Nous utiliserons donc une régression logistique L-BFGS (Limited memory Broyden–Fletcher–Goldfarb–Shanno). Plutôt que d'utiliser la méthode de Newton-Raphson pour la recherche des coefficients optimaux, nous utiliserons une méthode Quasi-Newton. L'algorithme se base sur une estimation de la matrice hessienne (méthode BFGS). L-BFGS, contrairement à BFGS, stocke cette estimation avec seulement quelques vecteurs (representation implicite). Ceci nous permet de gagner en mémoire et en rapidité. Et donc nous pouvons lancer notre algorithme (en précisant le nombre d'iterations) sur un très grand nombre de variables.
> 
*
In our case, we will use the logistic regression model. However, we have a very big variable's number, the variables are the words that we have stored. We will use the logistic regression model with L-BFGS (Limited memory Broyden–Fletcher–Goldfarb–Shanno). Instead of using Newton-Raphson algorithm to calculate the coefficients, we will use a Quasi-Newton algorithm. This algorithm is based on an estimation of the Hessian matrix (BFGS). On contrary to BFGS, L-BFGS stores only a few vectors (implicit representation). With this, we save memory and we are faster. Finally, we can run our algorithm (indicating the iterations' number) on a big number of variables.
*

## Importation des libraries - Library import

In [19]:
# data transformation
from pyspark.mllib.linalg import Vectors  
from pyspark.mllib.regression import LabeledPoint

# for analyzing the results
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.mllib.evaluation import BinaryClassificationMetrics

# for logistic regression
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.classification import LogisticRegressionModel

# for SVM (for comparison)
from pyspark.mllib.classification import SVMModel
from pyspark.mllib.classification import SVMWithSGD 

## Mise en forme du texte - Text formatting

Nous devons d'abord nettoyer le texte des accents, caractères spéciaux et autres.
> 
*First of all, we have to clean the text from any accents, specials characters and others.*

In [20]:
def nettoyage_texte(text) :
    texte = text.replace('À', 'A').replace('Á', 'A').replace('Â', 'A').replace('Ã', 'A') \
    .replace('È', 'E').replace('É', 'E').replace('Ê', 'E').replace('Ë', 'E') \
    .replace('Í', 'I').replace('Ì', 'I').replace('Î', 'I').replace('Ï', 'I') \
    .replace('Ù', 'U').replace('Ú', 'U').replace('Û', 'U').replace('Ü', 'U') \
    .replace('Ò', 'O').replace('Ó', 'O').replace('Ô', 'O').replace('Õ', 'O') \
    .replace('Ö', 'O').replace('Ñ', 'N').replace('Ç', 'C').replace('ª', 'A') \
    .replace('º', 'O').replace('§', 'S').replace('³', '3').replace('²', '2') \
    .replace('¹', '1').replace('à', 'a').replace('á', 'a').replace('â', 'a') \
    .replace('ã', 'a').replace('ä', 'a').replace('è', 'e').replace('é', 'e') \
    .replace('ê', 'e').replace('ë', 'e').replace('í', 'i').replace('ì', 'i') \
    .replace('î', 'i').replace('ï', 'i').replace('ù', 'u').replace('ú', 'u') \
    .replace('û', 'u').replace('ü', 'u').replace('ò', 'o').replace('ó', 'o') \
    .replace('ô', 'o').replace('õ', 'o').replace('ö', 'o').replace('ñ', 'n') \
    .replace('Ä', 'A').replace('ç', 'c') \
    .replace("!"," ").replace("."," ").replace("?"," ").replace(","," ") \
    .replace(";"," ").replace(":"," ").replace("/"," ").replace("+"," ") \
    .replace("%"," ").replace("("," ").replace(")"," ").replace("["," ") \
    .replace("]"," ").replace("&"," ").replace("`"," ").replace("*"," ") \
    .replace("$"," ").replace("«"," ").replace("»"," ").replace("'"," ") \
    .replace("_"," ").replace("\t"," ").replace("|"," ").replace("\""," ") \
    .replace("0"," ").replace("1"," ").replace("2"," ").replace("3"," ") \
    .replace("4"," ").replace("5"," ").replace("6"," ").replace("7"," ") \
    .replace("8"," ").replace("9"," ") \
    .replace("!"," ").replace("."," ").replace("?"," ").replace(","," ") \
    .replace(";"," ").replace(":"," ").replace("/"," ").replace("+"," ") \
    .replace("%"," ").replace("("," ").replace(")"," ").replace("["," ") \
    .replace("]"," ").replace("&"," ").replace("`"," ").replace("*"," ") \
    .replace("$"," ").replace("«"," ").replace("»"," ").replace("'"," ") \
    .replace("_"," ").replace("\t"," ").replace("|"," ").replace("\""," ") \
    .replace(" -"," ").replace("- "," ").replace("--"," ").replace(" - "," ") \
    .lower().strip().split()
    return texte

texte = nettoyage_texte("Ëxémplè : J'äï$ réùssî à néttöyer- et re- néttöyer môn 1 téxtè !")
print(texte)

['exemple', 'j', 'ai', 'reussi', 'a', 'nettoyer', 'et', 're', 'nettoyer', 'mon', 'texte']


Ensuite, il faut mettre en forme le texte pour que le modèle puisse lire les données
> *Then, we have to format the text so that the model can read the data*

In [21]:
def mise_en_forme(liste_mots, texte, sentiment = 1):
    data = zip(texte, [texte.count(w) for w in texte]) # Contage des mots du texte (effectifs)
    data = list(set(data))  # Retrait des doublons
    data = [w for w in data if (w[0],0) in liste_mots] # Filtre les mots retenus précédemment (dans liste_mots)
    # Ajout des mots retenus qui ne sont pas présents dans la phrase sous la forme: (mot,0)
    data = data +[ w for w in liste_mots if w[0] not in [x[0] for x in data] ]
    data = sorted(data, key=lambda l: l[0])  # On trie par ordre alphabétique des mots
    data = LabeledPoint( sentiment ,Vectors.dense([w[1] for w in data]) )
    return data

## Chargement de la base de données - Loading the database

In [22]:
donnees=sc.textFile("hdfs://ecoles.node1.pro.hupi.loc/user/anthony.laffond/CommentairesLivres5000_pr.csv",use_unicode=False) \
          .map( lambda l : ( l[0:(len(l)-10)] , l[(len(l)-10):len(l)] ) ) \
          .persist()
n= donnees.count()
donnees.first()

("Il n'est inutile de preciser que l'auteur est suisse, et qu'il ne s'agit pas d'une traduction, mais d'un roman noir, se deroulant aux etats-Unis, avec des codes tres etatsuniens, mais tournes en derision. Car, apres lecture d'autres commentaires, je trouve qu'on oublie de dire que, parfois, c'est tres drole (si, si, j'ai eclate de rire plusieurs fois en le lisant) - a d'autres moments, on est dans une enquete tres serieuse. Pour tout dire, j'ai parfois eu l'impression de me retrouver dans Fargo des freres Coen : tous les protagonistes se comportent tous a un moment ou a un autre comme de sombres cretins, meme si ce qui arrive est tragique. Harry Quebert n'y echappe pas tant on se demande bien comment il a pu tomber amoureux d'une Nola dont on peine a percevoir les qualites tout au long du roman (s'il devait y avoir un cote roman d'amour, de ce cote, c'est rate, mais ce n'etait peut-etre pas du tout le but...). Outre Fargo, une autre reference pourrait etre Twin Peaks. Le style a ete 

## Nettoyage de la base de données - Database cleaning

In [23]:
texte = donnees.map( lambda l : (nettoyage_texte(l[0]),nettoyage_texte(l[1])) ) \
               .map( lambda l : (l[0], (l[1]==["positive"])*1) ).persist() 
n= texte.count()
print(texte.first())

(['il', 'n', 'est', 'inutile', 'de', 'preciser', 'que', 'l', 'auteur', 'est', 'suisse', 'et', 'qu', 'il', 'ne', 's', 'agit', 'pas', 'd', 'une', 'traduction', 'mais', 'd', 'un', 'roman', 'noir', 'se', 'deroulant', 'aux', 'etats-unis', 'avec', 'des', 'codes', 'tres', 'etatsuniens', 'mais', 'tournes', 'en', 'derision', 'car', 'apres', 'lecture', 'd', 'autres', 'commentaires', 'je', 'trouve', 'qu', 'on', 'oublie', 'de', 'dire', 'que', 'parfois', 'c', 'est', 'tres', 'drole', 'si', 'si', 'j', 'ai', 'eclate', 'de', 'rire', 'plusieurs', 'fois', 'en', 'le', 'lisant', 'a', 'd', 'autres', 'moments', 'on', 'est', 'dans', 'une', 'enquete', 'tres', 'serieuse', 'pour', 'tout', 'dire', 'j', 'ai', 'parfois', 'eu', 'l', 'impression', 'de', 'me', 'retrouver', 'dans', 'fargo', 'des', 'freres', 'coen', 'tous', 'les', 'protagonistes', 'se', 'comportent', 'tous', 'a', 'un', 'moment', 'ou', 'a', 'un', 'autre', 'comme', 'de', 'sombres', 'cretins', 'meme', 'si', 'ce', 'qui', 'arrive', 'est', 'tragique', 'harry'

## Création d'un dictionnaire - Dictionnary creation

Nous avons besoin de créer un dictionnaire de mots qui définira nos variables
> 
*We have to create a dictionnary which will define our variables (features)*

In [24]:
# Occurence des mots sur la BD / Word Occurence on the DB
selection = sorted ( texte.flatMap(lambda l : l[0]).groupBy(lambda w : w) \
                 .map( lambda l : (l[0],len(l[1])) ).collect() )
print("nombre de mots sur l'ensemble des données : " + str(len(selection)) )
selection[25:30]

nombre de mots sur l'ensemble des données : 27218


[('abandon', 14),
 ('abandonnant', 3),
 ('abandonne', 39),
 ('abandonnee', 7),
 ('abandonnent', 2)]

In [25]:
# Importation de la base de StopWords / StopWords list import
stopwords = sc.textFile("hdfs://ecoles.node1.pro.hupi.loc/user/anthony.laffond/StopWordsCleanWN.txt",use_unicode=False).collect()
print("Nombre de StopWords : "+ str(len(stopwords)) )

Nombre de StopWords : 218


In [26]:
min_eff = 20
liste_mots = sorted([ (w[0],0) for w in selection if ( (w[1]>min_eff) and (w[0] not in stopwords) )])
print("Nombre de mots retenus : " + str(len(liste_mots)) )

Nombre de mots retenus : 2668


In [15]:
liste_mots[200:205]

[('an', 0), ('analyse', 0), ('analyser', 0), ('analyses', 0), ('ancien', 0)]

In [16]:
sc.parallelize(sorted([x[0] for x in liste_mots])) \
  .saveAsTextFile("hdfs://ecoles.node1.pro.hupi.loc/user/anthony.laffond/my_liste4700py")

## Mise en forme des données - Data formatting

In [17]:
data = texte.map( lambda l : mise_en_forme(liste_mots, l[0] ,l[1]) )
n_data = data.count()
print( "Taille de la base de données : "+str(n_data) )

Taille de la base de données : 5470


## Apprentissage/Validation - Training/Validation

On va découper notre base de données en un échantillon d'apprentissage (80% pour la construction du modèle) et un échantillon de test (20% pour le calcul d'indicateurs de qualité du modèle). 
> 
*We are going to split our database into a training dataset (80% to build the model) and a test dataset (20% to calculate features of model quality)*

In [18]:
splits = data.randomSplit([0.8, 0.2], seed = 1234)
training = splits[0].cache()
test = splits[1].cache()

n_training = training.count()
n_test = test.count()
print( "Taille de l'échantillon d'apprentissage : " + str(n_training) )
print( "Taille de l'échantillon de test : " + str(n_test) )

# Le découpage en 2 échantillons est plutôt long comparé à Scala / the split in 2 parts is longer than in Scala

Taille de l'échantillon d'apprentissage : 4342
Taille de l'échantillon de test : 1128


In [133]:
print( "Taux de positif dans la base de données : " + str(data.map(lambda l: l.label).sum()/float(n_data)) ) 
print( "Taux de positif dans l'échantillon d'apprentissage : " + str(training.map(lambda l: l.label).sum()/float(n_training) ) )
print( "Taux de positif dans l'échantillon de test : " + str(test.map(lambda l: l.label).sum()/float(n_test) ) )

Taux de positif dans la base de données : 0.731992687386
Taux de positif dans l'échantillon d'apprentissage : 0.730769230769
Taux de positif dans l'échantillon de test : 0.73670212766


## Regression logistique - logistic regression

In [136]:
model = LogisticRegressionWithLBFGS.train(training, numClasses=2, iterations=1000) # Training
model.clearThreshold() # permet d'obtenir le score (et non directement le sentiment attribué)
                       #  - it allows us to get the score and not the sentiment

## Calcul du taux d'erreur - Calculating the error rate

In [137]:
# Sur l'échantillon test - On the test dataset
scoreAndLabels = test.map(lambda l : ( (model.predict(l.features)>0.5)*1.0 , l.label) )

taux_p = scoreAndLabels.map(lambda l: ( (l[0]==0) and (l[1]==1) )*1 ).reduce(lambda x,y : x+y )
taux_n = scoreAndLabels.map(lambda l: ( (l[0]==1) and (l[1]==0) )*1 ).reduce(lambda x,y : x+y )
taux_pb = scoreAndLabels.map(lambda l: ( (l[0]==1) and (l[1]==1) )*1 ).reduce(lambda x,y : x+y )
taux_nb = scoreAndLabels.map(lambda l: ( (l[0]==0) and (l[1]==0) )*1 ).reduce(lambda x,y : x+y )
taux_err = scoreAndLabels.map(lambda l: (l[0]!=l[1])*1 ).reduce(lambda x,y : x+y )

print("Taux de positif mal prédit : " + str(taux_p/float(n_test)) )
print("Taux de négatif mal prédit : " + str(taux_n/float(n_test)) )
print("Taux de positif bien prédit : " + str(taux_pb/float(n_test)) )
print("Taux de négatif bien prédit : " + str(taux_nb/float(n_test)) )
print("Taux d'erreur : " + str(taux_err/float(n_test)) )

Taux de positif mal prédit : 0.0328014184397
Taux de négatif mal prédit : 0.0310283687943
Taux de positif bien prédit : 0.70390070922
Taux de négatif bien prédit : 0.232269503546
Taux d'erreur : 0.063829787234


## Calcul de l'AUC - Calculating the AUC

In [138]:
metrics = BinaryClassificationMetrics(scoreAndLabels)
AUC = metrics.areaUnderROC
print("Area under ROC = " + str(AUC) )

Area under ROC = 0.918815106541


## Sauvegarde du modèle - Saving the model

In [139]:
model.save(sc, "hdfs://ecoles.node1.pro.hupi.loc/user/anthony.laffond/model_reglog2600py")

In [141]:
# PMML : doesn't exist yet
#model.toPMML(sc,"hdfs://ecoles.node1.pro.hupi.loc/user/anthony.laffond/model_reglog_PMML2600py")